In [1]:
#import library 
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

In [2]:
#we need redshift jdbc jar file
jar_path = ''

In [3]:
#create spark session object
spark = SparkSession \
    .builder \
    .appName("integration_pyspark_redshift") \
    .config("spark.jars", jar_path)\
    .config('spark.driver.extraClassPath', jar_path) \
    .getOrCreate()

23/03/01 14:24:17 WARN Utils: Your hostname, ashwins-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.199 instead (on interface en0)
23/03/01 14:24:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/01 14:24:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 14:24:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/01 14:24:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
#provide aws credentials
spark._jsc.hadoopConfiguration().set("fs.s3.awsAccessKeyId", "")
spark._jsc.hadoopConfiguration().set("fs.s3.awsSecretAccessKey", "")

In [5]:
# create spark datafrme from csv file
AirlineDF = spark.read.option("header", "true").csv("airlines1.csv")

In [6]:
AirlineDF.show()

23/03/01 14:24:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/03/01 14:24:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, IATA_CODE_Reporting_Airline, Tail_Number, Flight_Number_Reporting_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsed

In [7]:
#create new dataframe and select some columns
AirlineDF1 = AirlineDF.select("Year","Reporting_Airline")

In [8]:
AirlineDF1.show()

+----+-----------------+
|Year|Reporting_Airline|
+----+-----------------+
|1998|               NW|
|2009|               FL|
|2013|               MQ|
|2010|               DL|
|2006|               US|
|1995|               DL|
|2006|               CO|
|2019|               9E|
|2008|               YV|
|2018|               WN|
|1991|               US|
|2014|               WN|
|1994|               AA|
|2013|               OO|
|2003|               UA|
|1988|               PI|
|2007|               NW|
|2015|               AS|
|2006|               UA|
|2017|               WN|
+----+-----------------+
only showing top 20 rows



In [9]:
k=AirlineDF1

# write dataframe into redshift

In [11]:
k.write \
  .format("jdbc") \
  .option("url","jdbc:redshift://redshift-cluster-2.cy9d1d0tbswp.ap-south-1.redshift.amazonaws.com:5439/dev?user=awsuser&password=letsetgo123A") \
  .option("Tempdir", "s3://airlines123/airline/") \
  .option("dbtable", "airline") \
  .mode("error") \
  .save()

23/03/01 14:28:49 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8
23/03/01 14:28:49 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8
23/03/01 14:28:49 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8
23/03/01 14:28:49 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8
23/03/01 14:28:49 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8


23/03/01 14:33:52 ERROR Executor: Exception in task 2.0 in stage 3.0 (TID 5)
com.amazon.redshift.util.RedshiftException: This connection has been closed.
	at com.amazon.redshift.jdbc.RedshiftConnectionImpl.checkClosed(RedshiftConnectionImpl.java:1026)
	at com.amazon.redshift.jdbc.RedshiftConnectionImpl.rollback(RedshiftConnectionImpl.java:1037)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.savePartition(JdbcUtils.scala:748)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1(JdbcUtils.scala:868)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1$adapted(JdbcUtils.scala:867)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2(RDD.scala:1011)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2$adapted(RDD.scala:1011)
	at org.apache.spark.SparkContext.$anonfun$runJob$5(SparkContext.scala:2268)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task

Py4JJavaError: An error occurred while calling o55.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 3.0 failed 1 times, most recent failure: Lost task 2.0 in stage 3.0 (TID 5) (192.168.29.199 executor driver): com.amazon.redshift.util.RedshiftException: This connection has been closed.
	at com.amazon.redshift.jdbc.RedshiftConnectionImpl.checkClosed(RedshiftConnectionImpl.java:1026)
	at com.amazon.redshift.jdbc.RedshiftConnectionImpl.rollback(RedshiftConnectionImpl.java:1037)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.savePartition(JdbcUtils.scala:748)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1(JdbcUtils.scala:868)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1$adapted(JdbcUtils.scala:867)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2(RDD.scala:1011)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2$adapted(RDD.scala:1011)
	at org.

23/03/01 14:33:55 WARN TaskSetManager: Lost task 0.0 in stage 3.0 (TID 3) (192.168.29.199 executor driver): TaskKilled (Stage cancelled)


23/03/01 14:33:55 WARN TaskSetManager: Lost task 1.0 in stage 3.0 (TID 4) (192.168.29.199 executor driver): TaskKilled (Stage cancelled)


23/03/01 14:34:03 WARN TaskSetManager: Lost task 3.0 in stage 3.0 (TID 6) (192.168.29.199 executor driver): TaskKilled (Stage cancelled)


# read data from redshift

In [13]:
df = spark.read.format("jdbc") \
  .option("url","jdbc:redshift://redshift-cluster-2.cy9d1d0tbswp.ap-south-1.redshift.amazonaws.com:5439/dev?user=awsuser&password=letsetgo123A") \
  .option("Tempdir", "s3://airlines123/airline/") \
  .option("dbtable", "airline") \
  .load()

In [14]:
df.show()

+----+-----------------+
|year|reporting_airline|
+----+-----------------+
|2007|               AA|
|1991|               DL|
|2004|               DL|
|1991|               US|
|1988|               UA|
|2018|               OH|
|2007|               US|
|2004|               HP|
|2001|               AA|
|2013|               WN|
|2016|               WN|
|2009|               DL|
|2007|               XE|
|2006|               CO|
|2016|               WN|
|1996|               US|
|2006|               EV|
|2003|               OO|
|2010|               EV|
|2007|               EV|
+----+-----------------+
only showing top 20 rows



# Read data from a query

In [15]:
df1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:redshift://redshift-cluster-2.cy9d1d0tbswp.ap-south-1.redshift.amazonaws.com:5439/dev?user=awsuser&password=letsetgo123A") \
    .option("query", "select count(*) from airline") \
    .option("tempdir", "s3://airlines123/airline/") \
    .load()

In [16]:
df1.show()

+-----+
|count|
+-----+
| 3800|
+-----+




[Stage 5:>                                                          (0 + 1) / 1]

